# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import json

# Load environment variables
_ = load_dotenv(find_dotenv())

# Get OpenAI API key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Function to call the OpenAI model
def return_CCRMSQL(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)

    new_context = context.copy()
    new_context.append({'role': 'user', 'content': "question: " + user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=new_context,
        temperature=0,
    )

    return response.choices[0].message.content

# Define database tables and structure
context = [ {'role': 'system', 'content': """
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(255)
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(255),
    Years DATE,
    Speciality VARCHAR(255),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);
"""} ]

# Add Few-Shot examples to improve response accuracy
context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient using valid SQLite syntax.

-- Example 1: Retrieve the name of the highest-paid employee.
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;

-- Example 2: Find the institution with the highest average salary.
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_usr = e.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
"""})

# Example user queries
queries = [
    "Which employee has the highest salary?",
    "What is the institution with the highest average salary?",
    "List all employees who have studied at a specific institution."
]

# Execute and print SQL responses
for query in queries:
    print(f"User Query: {query}")
    print("Generated SQL Query:")
    print(return_CCRMSQL(query, context))
    print("-" * 50)


User Query: Which employee has the highest salary?
Generated SQL Query:
To find the employee with the highest salary, you can run the following query:

```sql
SELECT e.name 
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This query will return the name of the employee who has the highest salary.
--------------------------------------------------
User Query: What is the institution with the highest average salary?
Generated SQL Query:
The institution with the highest average salary is retrieved using the following SQL query:

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary 
FROM studies st 
JOIN employees e ON st.ID_usr = e.ID_usr 
JOIN salary sa ON e.ID_usr = sa.ID_usr 
GROUP BY st.Institution 
ORDER BY avg_salary DESC 
LIMIT 1;
```

This query calculates the average salary for each institution based on the employees' salaries and then selects the institution with the highest average salary.
----------------------------------------

In [2]:
!pip install python-dotenv


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

Objective
This exercise aimed to compare two different prompt strategies for generating SQL queries using GPT-3.5-Turbo. The first (old) approach provided only table names and short descriptions, while the second (new) approach followed best practices outlined in a study from Ohio University. This included using SQL CREATE TABLE statements, sample data, and few-shot learning with example queries.

Approach
Dataset Setup:

Three tables were used in both methods:
employees (ID, name)
salary (ID, year, salary)
studies (ID, user ID, education level, institution, years, specialty)
Prompting Strategies:

Old Approach: Tables and attributes were described in plain text.
New Approach:
Used SQL CREATE TABLE statements for clarity.
Included sample data to provide context.
Provided few-shot learning examples to guide SQL generation.
Test Queries:

Find the highest-paid employee.
Retrieve the university with the highest average salary.
List employees with a master's degree who earn more than $60,000.
Findings
Query Accuracy & Improvements:

The new approach consistently generated more accurate SQL queries, with correct JOIN conditions and table references.
The old approach occasionally missed important joins or used incorrect table aliases.
Reduction in Hallucinations:

The old prompt sometimes included non-existent columns or tables, whereas the new prompt significantly reduced hallucinations by providing structured table definitions and examples.
Handling Complex Queries:

The new approach performed better with multi-table joins and aggregation queries, likely due to the example-based prompting.
Key Learnings
Providing SQL CREATE TABLE statements enhances query accuracy by giving GPT-3.5 a clearer database schema.
Few-shot learning significantly improves performance, especially for complex queries.
Reducing ambiguity in table names and relationships minimizes hallucinations.
Context size matters—longer prompts improve results but may limit model response length.
Conclusion
By structuring prompts using SQL CREATE TABLE statements, sample data, and example queries, GPT-3.5-Turbo becomes more reliable in generating SQL queries. This approach improves accuracy, reduces errors, and minimizes hallucinations, making it a best practice for text-to-SQL applications.





